In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


import pandas as pd
pd.options.plotting.backend = "plotly"
from plotly import express as ex
from tqdm import trange
import os

# LIFE Academy Exercise

This material draws upon, adapts and extends Lab 2 from MJ2380, authored by Roberto Heredia Fonseca, 
Shravan Kumar and Francesco Gardumi and the OpTIMUS.community 
and is licensed under the [Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).

This notebook provides a simple hands-on introduction to energy system modelling using OSeMOSYS.
We will run OSeMOSYS in the background using this Jupyter Notebook to control the input data, 
run the model and extract and visualise the results.

If you click on the **Jupyter** logo in the top left-hand corner of the screen, 
you will see the folder structure containing the OSeMOSYS models used in this lab.

In the `model` folder, you will find subfolders containing OSeMOSYS models of increasing complexity.
Each subfolder contains a data folder in which you see a number of CSV (comma-separated value) 
files which you can edit directly in the browser.  
Each file relates to one parameter in OSeMOSYS. 
For example, you could edit the `CapitalCost` of technologies of the `year` model by editing 
the respective [CSV file](model/year/data/CapitalCost.csv).

We use the multi-year model called `year`, which you can find in the [`model/year/data`](model/year/data) folder.

## Units

Here are the units used throughout the models

Parameter | Unit | Description
:-- | --: | --:
Demand | PJ | Petajoules
Capacity | GW | Gigawatts
Activity | PJ | Petajoules
Capital Cost | M\$/GW | Million US dollars per Gigawatt
Fixed Cost | M\$/GW | Million US dollars per Gigawatt
Variable Cost, Fossil Fuels Extraction/Import | M\$/PJ | Million US dollars per Petajoule
Variable Cost, Renewables | M\$/GWh | Million US dollars per Gigawatt-hour
Operational Life | Yr | Year
Emission Activity Ratio | MtCO2/PJ | Million tonnes of CO2 equivalent per petajoule
Annual Emission Limit | MtCO2 | Million tonnes of CO2 equivalent
Emission Penalty | \$/tCO2 | US dollars per tonne of CO2 equivalent

## Model Structure

### Sets

Sets are used to define the model structure. For example 
[YEAR](model/year/data/YEAR.csv), 
[TIMESLICE](model/year/data/TIMESLICE.csv), 
[TECHNOLOGY](model/year/data/TECHNOLOGY.csv), 
[MODE_OF_OPERATION](model/year/data/MODE_OF_OPERATION.csv), 
[REGION](model/year/data/REGION.csv), 
[FUEL](model/year/data/FUEL.csv), 
[EMISSION](model/year/data/EMISSION.csv) etc.

_Sets define the basic model structure_

### Parameters ("inputs")

Parameters define the numerical inputs to the model such as costs, demands, constraints for capacity etc.
Parameters are indexed over the sets, for example [`CapitalCost`](model/year/data/CapitalCost.csv). 
This represents an array of costs for every region, technology and year.

_Parameters are the inputs that the modeller needs to provide to the model._

**Open the [manual](https://osemosys.readthedocs.io/en/latest/manual/Structure%20of%20OSeMOSYS.html#parameters) in a browser window for reference throughout the exercise**

### Variables

The value of variables is decided by the solver, a piece of software used to "solve" the system of equations we use to define the energy system.
Broadly, variables are defined the investment decisions, and operational decisions. 
By changing the value of the decision variables, the solver can minimise the objective function.

Variables are also indexed over sets, for example: `Generation(Technology, Year)`. 
This represents an array of generation by every technology in every year

_Variables are the results that the modeller gets from the model_

## Running a model

Running an OSeMOSYS model using this Jupyter notebook is a two step process. 
Firstly, you need to create an OSeMOSYS datafile by running the following.

In [ ]:
!otoole convert csv datafile model/year/data temp.txt config.yaml


Note you need the `!` prepended to the command (this tells the notebook to run this command using the shell rather than Python). Here we create datafile called `temp.txt` from the model data stored in `model/year`.

After generating the OSeMOSYS datafile (in this case, called `temp.txt`) we then solve the model using `glpsol`, which is the open-source solver provided by the GNU Math Programming kit.


In [ ]:
!glpsol -d temp.txt -m osemosys.txt > osemosys.log

    
The results are then available in the `results` folder - see them [here](results/). You should also check the `osemosys.log` file to ensure that the model ran correctly.  View it [here](osemosys.log).

In the stages below, we load data from the model inputs and results using a Python library called `pandas` and plot them in interactive charts.

## Contents

- [Stage 1](#Stage-1:-Exploring-the-model) - In this section, we introduce and explore a simple energy system model.
- [Stage 2](#Stage-2:-A-new-technology) - We add a new technology to the model.
- [Stage 3](#Stage-3:-A-tax-on-pollution) - We add an emissions penalty, imposing a tax upon CO2

## Stage 1: Exploring the model

### Setting up the timeslices, model horizon and region

Production differs across the year - in OSeMOSYS we use `TIMESLICES` to represent the fractions of the year in which different levels of demand exist. This approximates the average demand profile for electricity and other fuels across days and seasons. In our example model, we have defined 6 time-slices:

TIMESLICE | Description
:--|:--
`ID` | Intermediate day
`IN` | Intermediate night
`SD` | Summer day
`SN` | Summer night
`WD` | Winter day
`WN` | Winter night

This example model only runs between 2022 and 2030. 
Note that this is not long enough to demonstrate what happens when 
a technology reaches the end of its lifetime, 
which is particularly important when you impose an emissions constraint.  
However, it does show how the model reacts to increasing demand.

This model uses a single region `SIMPLICITY` 
(which shows that this is an adaptation of an example model we use for teaching called Simplicity).

### The reference energy system

![Reference energy system for a ](img/gassolarwind.svg)

### Technologies and Commodities/Fuels

In this simple OSeMOSYS model, there are four natural gas technologies - imports of natural gas (`GasImport`), 
and extraction (`GasExtraction`, `GasExtraction2`, `GasExtraction3`). 
These technologies feed a natural gas combined-cycle electricity generation plant `NGCC`. 
This plant produces secondary electricity `SEC_EL` which enters a transmission and distribution technology `TD`. 
The transmission and distribution technology produces final electricity `FEL`.

There are also four coal extraction and import technologies, 
and these are linked via a coal fired generation plant (`SCC`) and the transmission and distribution technology `TD` 
in a similar manner to produce final electricity `FEL`.

This model also has a solar PV technology (`SOLPV`) and a wind power technology `WIND`.

The solar PV technology (`SOLPV`) and wind turbine technology (`WIND`) have the following characteristics:

Plant | Parameter | Value
:--|:--|--:
`SOLPV` | `CapitalCost` | 1700.0
`WIND` | `CapitalCost` | 1845.0
`SOLPV` | `VariableCost` | 2.5
`WIND`| `VariableCost` | 4.17
`SOLPV` | `OutputActivityRatio(FEL)` | 1.0
`WIND` | `OutputActivityRatio(SEC_EL)` | 1.0
`SOLPV` | `ResidualCapacity(2022)` | 2
`WIND`| `ResidualCapacity(2022)` | 3

Solar PV and Wind have the following capacity factors defined in the file `CapacityFactor.csv`:

Timeslice | SOLPV | WIND
:--|:--|--:
`ID` | 0.3 | 0.25
`IN` | 0.0 | 0.25
`SD` | 0.4 | 0.25
`SN` | 0.0 | 0.25
`WD` | 0.25 | 0.25
`WN` | 0.0 | 0.25

As you can see from the figure above, this model contains multiple gas resources, 
plus imports, multiple coal resources, plus an import, and a new super-critical coal fired power station.

The prices and quantities of the resources are as follows:

Resource | Quantity available | Cost of extraction/import
:--|--:|--:
`GasExtraction` | 10 | 8.0
`GasExtraction2` | 15 | 10.0
`GasExtraction3` | 30 | 11.0
`GasImport` | - | 12.0
`CoalExtraction` | 10 | 4.0
`CoalExtraction2` | 15 | 5.0
`CoalExtraction3` | 30 | 5.5
`CoalImport` | - | 12.0

These prices and quantities are defined in [`VariableCost.csv`](../edit/model/year/data/VariableCost.csv) and [`TotalTechnologyModelPeriodActivityUpperLimit.csv`](../edit/model/year/data/TotalTechnologyModelPeriodActivityUpperLimit.csv) respectively.

The gas-fired (`NGCC`) and coal-fired (`SCC`) power plants have the following characteristics:

Plant | Parameter | Value
:--|:--|--:
`NGCC` | `CapitalCost` | 1100.0
`SCC` | `CapitalCost` | 1600.0
`NGCC` | `FixedCost` | 44.0
`SCC` | `FixedCost` | 56.0
`NGCC` | `InputActivityRatio(GAS)` | 1.992
`SCC` | `InputActivityRatio(COA)` | 2.120
`NGCC` | `OutputActivityRatio(SEC_EL)` | 1.0
`SCC` | `OutputActivityRatio(SEC_EL)` | 1.0
`NGCC` | `ResidualCapacity` | 30
`SCC`| `ResidualCapacity` | 30


### Model representation of energy market

In OSeMOSYS, supply of energy must equal demand for energy. As the quantity of energy demanded increases, energy supply increases accordingly.
Demand is completely inelastic - the demand curve is a straight line (imagine a vertical line moving along the x-axis) 
so the equilibrium point (where the supply and demand curves cross) denotes the marginal cost of production and in this case is equal to the price.

Within most energy systems, there are multiple markets for different, related, energy commodities. 
We can use an OSeMOSYS model to understand the interactions between these different energy markets.

In OSeMOSYS, there is a key equation called a "balancing constraint". This ensures that energy is conserved in each timeslice. 

```ampl
s.t. EBa11_EnergyBalanceEachTS5{r in REGION, l in TIMESLICE, f in FUEL, y in YEAR}: 
	Production[r,l,f,y] >= Demand[r,l,f,y] + Use[r,l,f,y] 
	+ sum{rr in REGION} Trade[r,rr,l,f,y] * TradeRoute[r,rr,f,y];
```

We can extract some useful information from this equation when we solve the optimisation problem to minimise costs.
These data are stored in the result files [`ProductionDual.csv`](results/ProductionDual.csv) and [`ProductionDualAnnual.csv`](results/ProductionDualAnnual.csv).

## 1. Edit the Data

Try editing some of the model parameters and see what happens to the results of the model.

Start with:
1. Increasing or decreasing demand [`SpecifiedAnnualDemand`](model/year/data/SpecifiedAnnualDemand.csv)
2. Play with changing the [`CapitalCost`](model/year/data/CapitalCost.csv) of the generation technologies
3. Try making the fuels more expensive by changing [`VariableCost.csv`](../edit/model/year/data/VariableCost.csv)

## 2. Run The Model

In [ ]:
!otoole convert csv datafile model/year/data temp.txt config.yaml
!glpsol -d temp.txt -m osemosys.txt > osemosys.log

## 3. View the plots

The results are then available in the `results` folder - see them [here](results/). You should also check the `osemosys.log` file to ensure that the model ran correctly.  View it [here](osemosys.log).

In [ ]:
outputs = pd.read_csv('model/year/data/OutputActivityRatio.csv')
techs_sec_el = outputs[outputs['FUEL'].isin(['SEC_EL'])]['TECHNOLOGY'].unique().tolist()
primary_extraction = outputs[outputs['FUEL'].isin(['COA', 'GAS'])]['TECHNOLOGY'].unique().tolist()

### Costs

In [ ]:
total_cost = pd.read_csv('results/TotalDiscountedCost.csv')
total_cost.plot(x='YEAR', y='VALUE', title='Total Discounted Costs (M$)', kind='bar')

In [ ]:
print(f"Objective Function: $M {total_cost['VALUE'].sum():.2f}")

In [ ]:
capital_investment = pd.read_csv('results/CapitalInvestment.csv')
capital_investment.plot(x='YEAR', y='VALUE', color='TECHNOLOGY', 
                        kind='bar', title='Capital Investment by Technology (M$)')

### Demand

In [ ]:
demand = pd.read_csv('results/Demand.csv').groupby(['REGION', 'FUEL', 'YEAR']).sum(numeric_only=True).reset_index()
demand.plot(x='YEAR', y='VALUE', color='FUEL', title='Demand (PJ)')

### Total Capacity

In [ ]:
total_capacity = pd.read_csv('results/TotalCapacityAnnual.csv')
total_generation_capacity = total_capacity[total_capacity['TECHNOLOGY'].isin(techs_sec_el)]
total_extraction_capacity = total_capacity[total_capacity['TECHNOLOGY'].isin(primary_extraction)]

total_generation_capacity.plot(x='YEAR', y='VALUE', color='TECHNOLOGY', kind='bar', 
    title='Total Annual Generation Capacity (GW)')

### New Capacity

This plot shows new investments in generation technologies

In [ ]:
capacity = pd.read_csv('results/NewCapacity.csv')
new_generation_capacity = capacity[capacity['TECHNOLOGY'].isin(techs_sec_el)]
new_extraction_capacity = capacity[capacity['TECHNOLOGY'].isin(primary_extraction)]

In [ ]:
new_generation_capacity.plot(x='YEAR', y='VALUE', color='TECHNOLOGY', kind='bar', title='New Capacity (GW)')

### Accumulated New Capacity in Extraction and Import Technologies

In [ ]:
new_extraction_capacity.plot(x='YEAR', y='VALUE', color='TECHNOLOGY', kind='bar', title='Accumulated New Capacity in Extraction and Import Technologies (PJ)')

### Production of electricity

In [ ]:
production = pd.read_csv('results/ProductionByTechnologyAnnual.csv')
generation_production = production[production['TECHNOLOGY'].isin(techs_sec_el)].groupby(by=['TECHNOLOGY', 'FUEL', 'YEAR']).sum(numeric_only=True).reset_index()
extraction_production = production[production['TECHNOLOGY'].isin(primary_extraction)].groupby(by=['TECHNOLOGY', 'FUEL', 'YEAR']).sum(numeric_only=True)

generation_production.plot(x='YEAR', y='VALUE', color='TECHNOLOGY', kind='bar', title='Electricity Production (PJ)')

### Extraction Production

In [ ]:
extraction_production.reset_index().plot(x='YEAR', y='VALUE', color='TECHNOLOGY', kind='bar', title='Extraction (PJ)')

### Emissions

In [ ]:
emissions = pd.read_csv('results/AnnualEmissions.csv')
emissions.plot(x='YEAR', y='VALUE', title='CO2 emissions (MtCO2e)', kind='bar')

In [ ]:
tech_emissions = pd.read_csv('results/AnnualTechnologyEmission.csv')
tech_emissions.plot(x='YEAR', y='VALUE', color='TECHNOLOGY', kind='bar', title='CO2 Emissions by Technology (MtCO2e)')

# Stage 2: A new technology

To add a new technology into the OSeMOSYS model stored you will need to edit the following files: 

  - `InputActivityRatio.csv`, 
  - `OutputActivityRatio.csv`,
  - `CapacityFactor.csv`
  - `OperationalLife.csv`, 
  - `ResidualCapacity.csv`, 
  - `TECHNOLOGY.csv`, 
  - `FixedCost.csv` and 
  - `VariableCost.csv`.

# Stage 3: A tax on pollution

In this stage, we'll now implement a financial penalty for emission of CO2.  


### Adding an emissions penalty to OSeMOSYS

To impose a financial penalty in OSeMOSYS we can edit the [EmissionsPenalty](../edit/model/year/data/EmissionsPenalty.csv) parameter file. Units are \$/tCO2.

You'll need to add a new line to the parameter file like so for each year from 2022 to 2030:

    SIMPLICITY,CO2,2022,50
    SIMPLICITY,CO2,2023,50    
    etc.    
    
First try a value of $50/tCO2.

### Use an emissions cap instead

To enter an emissions cap in OSeMOSYS, edit the [ModelPeriodEmissionLimit.csv](../edit/model/year/data/ModelPeriodEmissionLimit.csv) parameter file.
For example, to specify that the only 20 MtCO2e are emitted between 2022 and 2030, enter:

    SIMPLICITY,CO2,20